### Add the scripts to the notebooks path

In [1]:
import os, sys

current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

scripts_path = os.path.join(parent_dir, 'scripts')

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

# Insert the path to the Scripts directory
sys.path.insert(0, scripts_path)

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

d:\KifiyaAIM-Course\Week - 8&9\Adey-Innovations-Fraud-Detection\notebooks


### Import Statements

In [2]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
sns.set_theme()

### Load the Data

In [4]:
# define path to the csv files
COUNTRY_IP_DATA = "../data/IpAddress_to_Country.csv"
FRAUD_DATA = "../data/Fraud_Data.csv"
CREDIT_DATA = "../data/creditcard.csv"

# load the data into pandas dataframes
country_ip_mapping = pd.read_csv(COUNTRY_IP_DATA)
fraud_data = pd.read_csv(FRAUD_DATA)
credit_data = pd.read_csv(CREDIT_DATA)

### Preprocess data

In [5]:
from scripts.feature_engineering import FeatureEngineering

1) First pass the data through the feature egineering pipeline. It contains the following processes.

merege ip and fraud data -> calculate transaction velocity and frequency -> break down date features -> handle missing values

In [6]:
data = FeatureEngineering.feature_enginering_pipeline(data=fraud_data, ip_mapping=country_ip_mapping)

2. Split the data into training and testing sets

In [7]:
from sklearn.model_selection import train_test_split
from scripts.utils import load_pickle, pickle_object

In [8]:
# the root path where to export/load pickeled objects from
EXPORT_PATH_ROOT = "../feature_store" 

First for the credit card data

In [9]:
# define features
credit_card_features = [column for column in credit_data.columns if column not in ["Class"]]

# define target 
credit_card_target = "Class"

# save the feature into pickle file
feature_save_path = os.path.join(EXPORT_PATH_ROOT, 'credit_features.pkl')
pickle_object(file_path=feature_save_path, object=credit_card_features)

# save the target into a pickle file
target_save_path = os.path.join(EXPORT_PATH_ROOT, 'credit_target.pkl')
pickle_object(file_path=target_save_path, object=credit_card_target)

In [10]:
# obtain the features and targets
credit_X = credit_data[credit_card_features]
credit_y = credit_data[credit_card_target]

# split them into training and testing features
credit_train, credit_test, credit_y_train, credit_y_test = train_test_split(credit_X, credit_y, test_size=0.3, random_state=7)

Now for the fraud data

In [11]:
# define features
fraud_features = [column for column in data.columns if column not in ["class", "user_id", "device_id"]]

# define targets
fraud_targets = "class"

# save the feature into pickle file
feature_save_path = os.path.join(EXPORT_PATH_ROOT, 'fraud_features.pkl')
pickle_object(file_path=feature_save_path, object=fraud_features)

# save the target into a pickle file
target_save_path = os.path.join(EXPORT_PATH_ROOT, 'fraud_target.pkl')
pickle_object(file_path=target_save_path, object=fraud_targets)

In [34]:
# obtain the features and targets
fraud_X = data[fraud_features]
fraud_y = data[fraud_targets]

# split them into training and testing features
fraud_train, fraud_test, fraud_y_train, fraud_y_test = train_test_split(fraud_X, fraud_y, test_size=0.3, random_state=7)

3. Normalize the numerical features

In [35]:
# the root path to store scalers and numerical encoders in
ENCODERS_PATH_ROOT = "../scalers"

First normalize the credit  scoring numerical features

In [36]:
# normalize the numerical features in the credit card data using the training data and save the scaler to be used during inference
credit_train, credit_numerical_scaler = FeatureEngineering.normalize_numerical_features(data=credit_train)
credit_test = credit_numerical_scaler.transform(X=credit_test)

# save the scaler
pickle_object(file_path=os.path.join(ENCODERS_PATH_ROOT, 'credit_scaler.pkl'), object=credit_numerical_scaler)

Now normalize the fraud numerical features

In [37]:
# normalize the numerical features in the fraud data using training data and save the scaler to be used during inference
fraud_train_numeric_scaled, fraud_numerical_scaler = FeatureEngineering.normalize_numerical_features(data=fraud_train)
scaled_columns = fraud_numerical_scaler.get_feature_names_out()
fraud_test_numeric_scaled = fraud_numerical_scaler.transform(X=fraud_test[scaled_columns])

# replace the numerical columns with the scaled ones
fraud_train[scaled_columns] = fraud_train_numeric_scaled[scaled_columns]
fraud_test[scaled_columns] = fraud_test_numeric_scaled

# save the numeric columns that are scaled into pickle files
pickle_object(file_path=os.path.join(EXPORT_PATH_ROOT, 'scaled_numerical_features.pkl'), object=scaled_columns)

# save the scaler
pickle_object(file_path=os.path.join(ENCODERS_PATH_ROOT, 'fraud_scaler.pkl'), object=fraud_numerical_scaler)

4. Now encode the categorical features

In [38]:
from scripts.utils import use_label_encoder

The credit data is skipped because all of its values are numerical data. So no need to encode it. 
Only the fraud data is going to have to categorical columns to be encoded

In [39]:
# encode categorical features using data from training, obtain the encoders for each categorical columns
fraud_train, categorical_encoder = FeatureEngineering.encode_categorical_data(data=fraud_train)

# encode categorical features of the testing data
for categorical_column in categorical_encoder:
    fraud_test[categorical_column] = use_label_encoder(data=fraud_test[categorical_column], encoder=categorical_encoder[categorical_column])

# save the categorical column encoders in pickle files
pickle_object(file_path=os.path.join(ENCODERS_PATH_ROOT, 'categorical_encoder.pkl'), object=categorical_encoder)

### Set up mlflow for tracking experimentation

In [40]:
from scripts.modeling import ModelingPipeline

In [19]:
pipeline = ModelingPipeline(
    x_train=fraud_train,
    x_test=fraud_test,
    y_train=fraud_y_train,
    y_test=fraud_y_test,
    tracking_uri="sqlite:///../mlflow_runs/mlflow.db"
)

Create an mlflow experiment

In [41]:
experiment_id = pipeline.create_experiment(experiment_name='Experiment_1')

Train the models

In [21]:
pipeline.train_models()

[I 2024-10-29 16:23:07,702] A new study created in memory with name: no-name-a423dab5-ebdc-49c9-8bc4-916de027c117
[I 2024-10-29 16:23:07,854] Trial 0 finished with value: 0.9068690166321084 and parameters: {'C': 13.18082849075773, 'max_iter': 479}. Best is trial 0 with value: 0.9068690166321084.
[I 2024-10-29 16:23:07,990] Trial 1 finished with value: 0.9068690166321084 and parameters: {'C': 1.3845414368352644e-05, 'max_iter': 191}. Best is trial 0 with value: 0.9068690166321084.
[I 2024-10-29 16:23:08,126] Trial 2 finished with value: 0.9068690166321084 and parameters: {'C': 0.0645037673749587, 'max_iter': 179}. Best is trial 0 with value: 0.9068690166321084.
[I 2024-10-29 16:23:08,266] Trial 3 finished with value: 0.9068690166321084 and parameters: {'C': 0.02120778833695094, 'max_iter': 396}. Best is trial 0 with value: 0.9068690166321084.
[I 2024-10-29 16:23:08,405] Trial 4 finished with value: 0.9068690166321084 and parameters: {'C': 74.61891686416675, 'max_iter': 212}. Best is tri

#### Finished Training Logistic Regressor ####
Best parameters for Logistic Regressor: {'C': 13.18082849075773, 'max_iter': 479}
Test accuracy with best parameters: 0.9068690166321084


[I 2024-10-29 16:23:12,411] Trial 0 finished with value: 0.9503242599373538 and parameters: {'max_depth': 15, 'min_samples_split': 4}. Best is trial 0 with value: 0.9503242599373538.
[I 2024-10-29 16:23:13,673] Trial 1 finished with value: 0.9461110866016677 and parameters: {'max_depth': 20, 'min_samples_split': 7}. Best is trial 0 with value: 0.9503242599373538.
[I 2024-10-29 16:23:15,168] Trial 2 finished with value: 0.9418096792694225 and parameters: {'max_depth': 26, 'min_samples_split': 8}. Best is trial 0 with value: 0.9503242599373538.
[I 2024-10-29 16:23:16,669] Trial 3 finished with value: 0.9415449772797458 and parameters: {'max_depth': 25, 'min_samples_split': 5}. Best is trial 0 with value: 0.9503242599373538.
[I 2024-10-29 16:23:16,983] Trial 4 finished with value: 0.9549786032558345 and parameters: {'max_depth': 4, 'min_samples_split': 8}. Best is trial 4 with value: 0.9549786032558345.
[I 2024-10-29 16:23:18,068] Trial 5 finished with value: 0.9480522345259629 and parame

#### Finished Training Decision Tree ####
Best parameters for Decision Tree: {'max_depth': 4, 'min_samples_split': 8}
Test accuracy with best parameters: 0.9549786032558345


[I 2024-10-29 16:23:32,186] Trial 0 finished with value: 0.9549786032558345 and parameters: {'n_estimators': 92, 'max_depth': 11}. Best is trial 0 with value: 0.9549786032558345.
[I 2024-10-29 16:23:39,865] Trial 1 finished with value: 0.9549786032558345 and parameters: {'n_estimators': 292, 'max_depth': 2}. Best is trial 0 with value: 0.9549786032558345.
[I 2024-10-29 16:23:47,389] Trial 2 finished with value: 0.9549786032558345 and parameters: {'n_estimators': 74, 'max_depth': 9}. Best is trial 0 with value: 0.9549786032558345.
[I 2024-10-29 16:24:30,724] Trial 3 finished with value: 0.9549786032558345 and parameters: {'n_estimators': 191, 'max_depth': 31}. Best is trial 0 with value: 0.9549786032558345.
[I 2024-10-29 16:24:50,750] Trial 4 finished with value: 0.9549786032558345 and parameters: {'n_estimators': 173, 'max_depth': 11}. Best is trial 0 with value: 0.9549786032558345.
[I 2024-10-29 16:24:56,879] Trial 5 finished with value: 0.9549786032558345 and parameters: {'n_estimato

...

3306/3306 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9570 - loss: 0.1583 - val_accuracy: 0.9550 - val_loss: 0.1624


2024/10/29 16:31:47 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: Cannot log input example or model signature for input with type <class 'pandas.core.frame.DataFrame'>. TensorFlow Keras autologging can only log input examples and model signatures for the following input types: numpy.ndarray, dict[string -> numpy.ndarray], tensorflow.keras.utils.Sequence, and tensorflow.data.Dataset (TensorFlow >= 2.1.0 required)
2024/10/29 16:31:47 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/29 16:31:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


1417/1417 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


2024/10/29 16:31:53 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/29 16:31:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


#### Finished Training LSTM ####
Test accuracy: 0.9549786032558345
#### Finished Training All Models ####


### Download the best model from mlflow

In [42]:
pipeline.log_best_model(export_path='../model')